# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298522145482                   -0.85    5.1
  2   -8.300204580160       -2.77       -1.25    1.0
  3   -8.300443725427       -3.62       -1.89    2.6
  4   -8.300459762411       -4.79       -2.72    2.4
  5   -8.300463350372       -5.45       -3.01   11.1
  6   -8.300463939636       -6.23       -3.16    2.6
  7   -8.300464310876       -6.43       -3.31    1.0
  8   -8.300464465130       -6.81       -3.42    1.2
  9   -8.300464615258       -6.82       -3.69    1.4
 10   -8.300464634406       -7.72       -3.88    1.0
 11   -8.300464639990       -8.25       -4.12    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67465646597                   -0.70    6.8
  2   -16.67867232878       -2.40       -1.14    1.0
  3   -16.67921982255       -3.26       -1.87    3.0
  4   -16.67928056858       -4.22       -2.76    4.4
  5   -16.67928602520       -5.26       -3.18    5.2
  6   -16.67928620715       -6.74       -3.48    3.6
  7   -16.67928621936       -7.91       -3.94    1.0
  8   -16.67928622387       -8.35       -4.56    1.4


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32585813159                   -0.56    6.6
  2   -33.33274255951       -2.16       -1.00    1.1
  3   -33.33409607556       -2.87       -1.74    4.4
  4   -33.33429780758       -3.70       -2.64    2.8
  5   -33.33615019137       -2.73       -2.50    6.1
  6   -33.33694341769       -3.10       -2.52    8.2
  7   -33.33694338856   +   -7.54       -2.53    2.2
  8   -33.33679857875   +   -3.84       -2.26    2.8
  9   -33.33676314519   +   -4.45       -2.21    1.0
 10   -33.33683120541       -4.17       -2.32    1.0
 11   -33.33684889701       -4.75       -2.36    1.0
 12   -33.33686629562       -4.76       -2.40    1.0
 13   -33.33688813257       -4.66       -2.47    1.0
 14   -33.33692823163       -4.40       -2.75    3.2
 15   -33.33693490382       -5.18       -2.87    1.5
 16   -33.33694360107       -5.06       -3.59    3.5
 17   -33.33694392220       -6.49       -4.29 

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.297942087867                   -0.85    5.8
  2   -8.300189932026       -2.65       -1.59    1.8
  3   -8.300429987028       -3.62       -2.42    3.8
  4   -8.300319030715   +   -3.95       -2.17   12.0
  5   -8.300464512105       -3.84       -3.66    3.1
  6   -8.300464600674       -7.05       -3.96    3.0
  7   -8.300464627335       -7.57       -4.14    1.6


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.30495460818                   -0.56    6.8
  2   -33.32703663157       -1.66       -1.27    1.2
  3   -24.47262383857   +    0.95       -0.59    6.8
  4   -33.00139027453        0.93       -1.28    5.9
  5   -33.23565289277       -0.63       -1.28    5.5
  6   -33.29603513897       -1.22       -1.73    3.6
  7   -32.90786426868   +   -0.41       -1.25    4.8
  8   -33.33344061029       -0.37       -2.17    4.9
  9   -33.33310125159   +   -3.47       -2.18    2.2
 10   -33.33495825747       -2.73       -2.36    1.8
 11   -33.33644552033       -2.83       -2.65    1.8
 12   -33.33686779714       -3.37       -3.01    3.4
 13   -33.33693907908       -4.15       -3.41    3.8
 14   -33.33694320992       -5.38       -3.75    3.4
 15   -33.33693949011   +   -5.43       -3.71    3.9
 16   -33.33694388537       -5.36       -4.49    3.0


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.